<a href="https://colab.research.google.com/github/fatmabenhlel1/Few-Shot-Learning-/blob/main/Copie_de_C3D_training_vocal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import cv2
import random
import numpy as np
import datetime
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv3D, MaxPool3D, Flatten, Dense, Dropout, Activation
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import GlobalAveragePooling3D, GlobalMaxPooling3D, Concatenate

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os

def count_folders(directory):
    return len([name for name in os.listdir(directory) if os.path.isdir(os.path.join(directory, name))])

# Exemple d'utilisation

directory3 = "/content/drive/MyDrive/Database/data"

print(f"Nombre de dossiers : {count_folders(directory3)}")


Nombre de dossiers : 7247


In [4]:
def extract_label(folder_name):
    """Extrait le label après le deuxième underscore."""
    parts = folder_name.split("_")
    if len(parts) >= 3:
        return int(parts[2])  # 0: Pt, 1: 72, 2: 1
    else:
        raise ValueError(f"Impossible d'extraire le label depuis : {folder_name}")


def create_txt_list(data_dir, output_txt):
    samples = []

    for video_folder in os.listdir(data_dir):
        label = extract_label(video_folder)
        video_path = os.path.join("data", video_folder)  # important : relatif à "data"
        samples.append(f"{video_path} 1 {label}\n")



    with open(output_txt, "w") as f:
        f.writelines(samples)

    print(f"Fichier généré avec {len(samples)} vidéos : {output_txt}")

# Usage
create_txt_list("/content/drive/MyDrive/Database/data", "all.txt")


Fichier généré avec 7247 vidéos : all.txt


In [5]:
def c3d_backbone():
    input_shape = (112, 112, 10, 3)
    weight_decay = 0.003
    nb_classes = 4

    inputs = Input(input_shape)
    x = Conv3D(64, (3, 3, 3), strides=(1, 1, 1), padding='same',
               activation='relu', kernel_regularizer=l2(weight_decay))(inputs)
    x = MaxPool3D((2, 2, 1), strides=(2, 2, 1), padding='same')(x)

    x = Conv3D(128, (3, 3, 3), strides=(1, 1, 1), padding='same',
               activation='relu', kernel_regularizer=l2(weight_decay))(x)
    x = MaxPool3D((2, 2, 2), strides=(2, 2, 2), padding='same')(x)

    x = Conv3D(128, (3, 3, 3), strides=(1, 1, 1), padding='same',
               activation='relu', kernel_regularizer=l2(weight_decay))(x)
    x = MaxPool3D((2, 2, 2), strides=(2, 2, 2), padding='same')(x)

    x = Conv3D(256, (3, 3, 3), strides=(1, 1, 1), padding='same',
               activation='relu', kernel_regularizer=l2(weight_decay))(x)
    x = MaxPool3D((2, 2, 2), strides=(2, 2, 2), padding='same')(x)

    x = Conv3D(256, (3, 3, 3), strides=(1, 1, 1), padding='same',
               activation='relu', kernel_regularizer=l2(weight_decay))(x)
    x = MaxPool3D((2, 2, 2), strides=(2, 2, 2), padding='same')(x)




    model = Model(inputs, x)
    return model


In [6]:
def video_embedding_model():
    base = c3d_backbone()
    inputs = base.input
    feature_map = base.output

    # Pooling
    avg_pool = GlobalAveragePooling3D()(feature_map)
    max_pool = GlobalMaxPooling3D()(feature_map)

    # Combine both
    combined = Concatenate()([avg_pool, max_pool])  # Final shape: (1 x 2D)
    model = Model(inputs, combined)
    return model

In [14]:
def generator_train_batch(train_file, batch_size, num_classes, img_path):
    with open(train_file, "r") as f:
        lines = f.readlines()

    while True:
        for i in range(0, len(lines), batch_size):
            batch_lines = lines[i:i+batch_size]
            X_batch = []

            for line in batch_lines:
                try:
                    parts = line.strip().split()
                    if len(parts) < 3:
                        print(f"Malformed line skipped: {line.strip()}")
                        continue

                    path = parts[0]
                    folder_path = os.path.join(img_path, path)
                    frame_files = sorted(os.listdir(folder_path))

                    if len(frame_files) < 10:
                        print(f"Not enough frames in: {folder_path}")
                        continue

                    frames = []
                    for frame_file in frame_files[:10]:
                        img_path_full = os.path.join(folder_path, frame_file)
                        img = cv2.imread(img_path_full)

                        if img is None:
                            print(f"Could not read image: {img_path_full}")
                            raise ValueError("Image is None")

                        img = cv2.resize(img, (112, 112))
                        img = img / 255.0
                        frames.append(img)

                    frames = np.array(frames)
                    frames = np.transpose(frames, (1, 2, 0, 3))  # (112, 112, 10, 3)
                    X_batch.append(frames)

                except Exception as e:
                    print(f"Error processing {line.strip()}: {e}")
                    continue

            if len(X_batch) == 0:
                continue  # Skip if no valid samples in this batch

            X_batch = np.array(X_batch)

            # Dummy labels for feature extraction training (adjust size if needed)
            dummy_labels = np.zeros((len(X_batch), 512))  # match model output shape

            yield X_batch, dummy_labels



In [15]:
def load_frames_from_folder(folder_path):
    frames = []
    for frame_name in sorted(os.listdir(folder_path)):
        frame_path = os.path.join(folder_path, frame_name)
        img = cv2.imread(frame_path)
        img = cv2.resize(img, (112, 112))
        frames.append(img)
    frames = np.stack(frames, axis=1)  # (H, W, D, C) devient (H, D, W, C)
    return frames


In [16]:
class TimeHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        self.epoch_times = []

    def on_epoch_begin(self, epoch, logs=None):
        self.start_time = datetime.datetime.now()

    def on_epoch_end(self, epoch, logs=None):
        duration = (datetime.datetime.now() - self.start_time).total_seconds()
        self.epoch_times.append(duration)
        print(f" Temps pour l'epoch {epoch + 1}: {duration:.2f} secondes")


In [19]:
import os
import datetime
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf

def main():
    # === Chemins ===
    img_path = "/content/drive/MyDrive/Database/"  # correct path
    train_file = "all.txt"  # pas /train_list.txt

    # === Lire les fichiers pour compter les exemples ===
    with open(train_file, "r") as f:
        train_samples = len(f.readlines())

    # === Paramètres ===
    num_classes = 4
    batch_size = 8
    epochs = 25

    # === Création du modèle ===
    model = video_embedding_model()
    model.compile(loss="categorical_crossentropy", optimizer="adam")

    model.summary()

    # === Création dossier checkpoints si nécessaire ===
    os.makedirs("checkpoints", exist_ok=True)

    # === TensorBoard ===
    logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

    # === Checkpoint Callback to save after each epoch ===
    checkpoint_callback = ModelCheckpoint(
        filepath="checkpoints/epoch_{epoch:02d}.h5",
        save_freq='epoch',
        save_weights_only=False,
        save_best_only=False,
        verbose=1
    )

    # === Time Callback ===
    time_callback = TimeHistory()

    # === Entraînement ===
    history = model.fit(
        generator_train_batch(train_file, batch_size, num_classes=num_classes, img_path=img_path),
        steps_per_epoch=122,
        epochs=epochs,
        callbacks=[tensorboard_callback, checkpoint_callback, time_callback],
        verbose=1
    )

    # === Sauvegarde finale ===
    model.save("c3d_feature_extractor.h5")
    print("Feature extractor saved as c3d_feature_extractor.h5")




In [20]:
if __name__ == "__main__":
    main()


Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 112, 112,  │          0 │ -                 │
│ (InputLayer)        │ 10, 3)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_15 (Conv3D)  │ (None, 112, 112,  │      5,248 │ input_layer_3[0]… │
│                     │ 10, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling3d_15    │ (None, 56, 56,    │          0 │ conv3d_15[0][0]   │
│ (MaxPooling3D)      │ 10, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_16 (Conv3D)  │ (None, 56, 56,    │    221,312 │ max_pooling3d_15… │
│                     │ 10, 128)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling3d_16    │ (None, 28, 28, 5, │          0 │ conv3d_16[0][0]   │
│ (MaxPooling3D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_17 (Conv3D)  │ (None, 28, 28, 5, │    442,496 │ max_pooling3d_16… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling3d_17    │ (None, 14, 14, 3, │          0 │ conv3d_17[0][0]   │
│ (MaxPooling3D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_18 (Conv3D)  │ (None, 14, 14, 3, │    884,992 │ max_pooling3d_17… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling3d_18    │ (None, 7, 7, 2,   │          0 │ conv3d_18[0][0]   │
│ (MaxPooling3D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_19 (Conv3D)  │ (None, 7, 7, 2,   │  1,769,728 │ max_pooling3d_18… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling3d_19    │ (None, 4, 4, 1,   │          0 │ conv3d_19[0][0]   │
│ (MaxPooling3D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 256)       │          0 │ max_pooling3d_19… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 256)       │          0 │ max_pooling3d_19… │
│ (GlobalMaxPooling3… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_3       │ (None, 512)       │          0 │ global_average_p… │
│ (Concatenate)       │                   │            │ global_max_pooli… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,323,776 (12.68 MB)

 Trainable params: 3,323,776 (12.68 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/25
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 21s/step - loss: 0.4088 
Epoch 1: saving model to checkpoints/epoch_01.h5


 Temps pour l'epoch 1: 2566.93 secondes
122/122 ━━━━━━━━━━━━━━━━━━━━ 2567s 21s/step - loss: 0.4064
Epoch 2/25
  1/122 ━━━━━━━━━━━━━━━━━━━━ 40:41 20s/step - loss: 3.1585e-06Not enough frames in: /content/drive/MyDrive/Database/data/Pt_12_0_seq41
  4/122 ━━━━━━━━━━━━━━━━━━━━ 39:45 20s/step - loss: nan       

KeyboardInterrupt: 

In [31]:
import os
import cv2
import numpy as np
import random
import tensorflow as tf
from sklearn.metrics import classification_report

# Load model
model = tf.keras.models.load_model("/content/checkpoints/epoch_01.h5", compile=False)

# Dataset directory structure:
# dataset_root/
#   └── ClassName/
#         └── clip0/
#         └── clip1/
def load_dataset(dataset_dir, input_shape=(112, 112), max_frames=10):
    data = {}
    for class_name in sorted(os.listdir(dataset_dir)):
        class_path = os.path.join(dataset_dir, class_name)
        if not os.path.isdir(class_path):
            continue
        clips = []
        for clip_name in os.listdir(class_path):
            clip_path = os.path.join(class_path, clip_name)
            frames = []
            for frame_name in sorted(os.listdir(clip_path))[:max_frames]:
                frame_path = os.path.join(clip_path, frame_name)
                img = cv2.imread(frame_path)
                if img is None:
                    continue
                img = cv2.resize(img, input_shape)
                img = img / 255.0
                frames.append(img)
            if len(frames) == max_frames:
                frames = np.array(frames)
                frames = np.transpose(frames, (1, 2, 0, 3))
                clips.append(frames)
        if len(clips) >= 2:
            data[class_name] = clips[:2]  # only use clip0 and clip1
    return data

def extract_features(model, clips):
    return model.predict(np.array(clips), verbose=0)

def run_episodes(data, model, num_episodes=30):
    y_true_all = []
    y_pred_all = []
    class_names = list(data.keys())

    for episode in range(num_episodes):
        support_set = []
        support_labels = []
        query_set = []
        query_labels = []

        for idx, class_name in enumerate(class_names):
            clips = data[class_name]
            random.shuffle(clips)
            support_clip = clips[0]
            query_clip = clips[1]

            support_set.append(support_clip)
            support_labels.append(idx)
            query_set.append(query_clip)
            query_labels.append(idx)

        # Feature extraction
        support_features = extract_features(model, support_set)
        query_features = extract_features(model, query_set)

        # Nearest neighbor classification
        predictions = []
        for query_feat in query_features:
            dists = [np.linalg.norm(query_feat - supp_feat) for supp_feat in support_features]
            pred = np.argmin(dists)
            predictions.append(pred)

        y_true_all.extend(query_labels)
        y_pred_all.extend(predictions)

        print(f"Episode {episode+1}/{num_episodes} done")

    return y_true_all, y_pred_all, class_names

if __name__ == "__main__":
    dataset_dir = "/content/drive/MyDrive/Database/clips_fsl_v2"  # Replace with your path
    data = load_dataset(dataset_dir)

    y_true, y_pred, class_names = run_episodes(data, model, num_episodes=30)
    print("\n=== Classification Report ===")
    print(classification_report(y_true, y_pred, target_names=class_names))


Episode 1/30 done
Episode 2/30 done
Episode 3/30 done
Episode 4/30 done
Episode 5/30 done
Episode 6/30 done
Episode 7/30 done
Episode 8/30 done
Episode 9/30 done
Episode 10/30 done
Episode 11/30 done
Episode 12/30 done
Episode 13/30 done
Episode 14/30 done
Episode 15/30 done
Episode 16/30 done
Episode 17/30 done
Episode 18/30 done
Episode 19/30 done
Episode 20/30 done
Episode 21/30 done
Episode 22/30 done
Episode 23/30 done
Episode 24/30 done
Episode 25/30 done
Episode 26/30 done
Episode 27/30 done
Episode 28/30 done
Episode 29/30 done
Episode 30/30 done

=== Classification Report ===
                     precision    recall  f1-score   support

          Carcinoma       1.00      1.00      1.00        30
   Extreme_polipoid       1.00      1.00      1.00        30
         Laryngitis       0.65      1.00      0.79        30
Vocal_insufficiency       0.68      1.00      0.81        30
        leukoplacia       1.00      1.00      1.00        30
          papilloma       1.00      1.00 

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
